In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time
import plotting

import sys

from pathlib import Path
import neuroplotlib as nplt

%matplotlib notebook

In [ ]:
# !rm -r x86_64
# !nrnivmodl mechanisms
# mv mechanisms/x86_64 .

In [ ]:
probe = model.define_electrode(probe_type="linear")

In [ ]:
cell_ais = model.create(morph_modifier="hillock", release=False, l_hillock=20)

param_names = [param.name for param in cell_ais.params.values() if not param.frozen]

fitness_protocols_ais = evaluator.define_protocols(
    cell_ais,
    feature_set=None, 
    feature_file=None, 
    electrode=probe,
    protocols_with_lfp=None
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim_ais = ephys.simulators.LFPySimulator(cell_ais, cvode_active=True, electrode=probe)

eva_ais = ephys.evaluators.CellEvaluator(                    
        cell_model=cell_ais,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols_ais,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim_ais)

In [ ]:
cell_replace = model.create(morph_modifier="", release=False, l_hillock=20)

param_names = [param.name for param in cell_replace.params.values() if not param.frozen]

fitness_protocols_replace = evaluator.define_protocols(
    cell_replace,
    feature_set=None, 
    feature_file=None, 
    electrode=probe,
    protocols_with_lfp=None
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim_replace = ephys.simulators.LFPySimulator(cell_replace, cvode_active=True, electrode=probe)

eva_replace = ephys.evaluators.CellEvaluator(                    
                cell_model=cell_replace,                                                       
                param_names=param_names,                                                    
                fitness_protocols=fitness_protocols_replace,                                        
                fitness_calculator=fitness_calculator,                                      
                sim=sim_replace)

In [ ]:
release_params = evaluator.get_release_params()

In [ ]:
release_params

In [ ]:
for param in release_params.keys():
    if "axon_initial_segment" in param:
        if "Na12bar" in param:
            release_params[param] = 3.147 #release_params[param]*100
        if "Na16bar" in param:
            release_params[param] = 2 #release_params[param]*100
        if "kbar" in param:
            release_params[param] = 0.1
        if "g_pas" in param:
            release_params[param] = 4.7e-5
        if "e_pas" in param:
            release_params[param] = -79.6
        if "e_k" in param:
            release_params[param] = -85

In [ ]:
release_params

In [ ]:
t_start = time.time()
responses_ais = eva_ais.run_protocols(protocols=fitness_protocols_ais.values(), param_values=release_params)
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
t_start = time.time()
responses_replace = eva_replace.run_protocols(protocols=fitness_protocols_replace.values(), param_values=release_params)
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
# cell.freeze({})
# cell.instantiate(sim=sim)
# for sec in cell.LFPyCell.allseclist:
#     print(sec.name())
# cell.destroy()

In [ ]:
responses_ais.keys()

In [ ]:
plt.figure()
plt.plot(responses_ais["Step1.soma.v"]["time"], responses_ais["Step1.soma.v"]["voltage"], label="soma")
plt.plot(responses_ais["Step1.ais0.v"]["time"], responses_ais["Step1.ais0.v"]["voltage"], label="AIS proximal")
plt.plot(responses_ais["Step1.ais1.v"]["time"], responses_ais["Step1.ais1.v"]["voltage"], label="AIS distal")
plt.legend()

In [ ]:
plotting.plot_responses(responses_replace)

In [ ]:
cell_ais.seclist_names

In [ ]:
plotting.plot_responses(responses_ais)

In [ ]:
ax = plotting.plot_eap(responses_ais, fitness_protocols, protocol_name="Step1", probe=probe.probe, norm=True)
ax = plotting.plot_eap(responses_replace, fitness_protocols, protocol_name="Step1", probe=probe.probe, 
                       norm=True, ax=ax, color='C1')

In [ ]:
cell.freeze(release_params)
cell.instantiate(sim=sim)
nplt.plot_detailed_neuron(cell.LFPyCell, color_ais="r", color_hillock="k", color_myelin="g")
cell.destroy(sim=sim)
cell.unfreeze(release_params.keys())

In [ ]:
cell.icell.axonal

In [ ]:
for seg in sec.allseg():
    print(seg)

In [ ]:
cell.destroy(sim=sim)
cell.unfreeze(release_params.keys())

In [ ]:
cell.axonal

In [ ]:
nplt.plot_detailed_neuron(morphology="morphology/cell1.asc", color_axon="r")